<a href="https://colab.research.google.com/github/SolNam-UI/CTD/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import 하는 곳

In [13]:
pip install tensorflow

In [14]:
import tensorflow as tf
import turtle
import numpy as np

In [24]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/파이썬 프로젝트 CTD"
!git clone https://github.com/SolNam-UI/CTD.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/파이썬 프로젝트 CTD
Cloning into 'CTD'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [16]:
from google.colab import drive
drive.mount('/content/CTD/storage')

%cd /content/CTD/storage


Drive already mounted at /content/CTD/storage; to attempt to forcibly remount, call drive.mount("/content/CTD/storage", force_remount=True).
/content/CTD/storage


Turtle로 개, 고양이 인풋 받기 (동렬)

Turtle로 데이터 타입 인풋 받기 (동렬)

CNN으로 인풋 데이터 훈련하기(필요에 따라) (윤성)

데이터 저장 (동렬)

Turtle로 출력하기 (동렬)